In [1]:
import cv2            # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm_notebook #from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion
from lxml import etree
import xml.etree.ElementTree as ET
import random

#D:/cnntry/air_data/trainairplane/  #training pictures of airplane
#D:/cnntry/air_data/annot_train    #annotation files of training airplane pictures

#D:/cnntry/air_data/valairplane/
#D:/cnntry/air_data/annot_val/


TRAIN_dir = 'D:/cnntry/air_data/trainairplane'
TRAIN_annot_dir = 'D:/cnntry/air_data/annot_train'
TRAIN_DIR = 'D:\cnntry\\train'

#TEST_DIR = 'D:\cnntry\\test'
TEST_DIR ='andrej/datasets/cats-and-dogs/1/home/test'

DATA_file= 'train_airdata.npy' 
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'cnnv1-{}-{}.model'.format(LR, 'idk') # just so we remember which saved model is which, sizes must match

ModuleNotFoundError: No module named 'cv2'

In [ ]:
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]


def get_label(img):
    annot_label = img.split('.')[0]+'.xml'           #xml file name
    annot_xml = os.path.join(TRAIN_annot_dir,annot_label)  #xml file full path
    
    root = etree.parse(annot_xml).getroot()
    #for child in root: print(child.tag, child.attrib)
    count = sum(1 for _ in root.iter("object")) #count of objects in xml file
    a = random.getrandbits(1)
    if count == 1 : return [1,0]  # if only 1 airplane
    else: return [0,1]            # of more than one airplane
    
def create_data(DIR,savename):
    training_data = []
    for img in tqdm_notebook(os.listdir(DIR)):
        label = get_label(img)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save(savename, training_data)
    return training_data

def create_train_data():
    training_data = []
    for img in tqdm_notebook(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [ ]:
#TRAIN_DIR = '/floyd/input/cats_and_dogs/train' #'/train/train.csv' 
#print(os.listdir("/floyd/input/cats_and_dogs"))

train_data = create_data(TRAIN_dir,DATA_file)
#train_data = create_train_data()  
#print(os.listdir(TRAIN_dir)[0].split('.')[0])
#train_data = np.load('D:\cnntry\\train_data.npy')

In [ ]:
train_data = np.load('D:\cnntry\\train_airdata.npy')

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
tf.reset_default_graph()

In [ ]:
#propert structure
#leaky relu
#input normalization
#proper output
#reading output
#getting nice anchors https://machinethink.net/blog/object-detection/
decay=0.0005

tflearn.layers.conv.conv_2d (incoming, nb_filter, filter_size, strides=1, padding='same', 
                             activation='linear', bias=True, weights_init='uniform_scaling', 
                             bias_init='zeros', regularizer=None, weight_decay=0.001, trainable=True, 
                             restore=True, reuse=False, scope=None, name='Conv2D')

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

filters=[16,32,64,128,256,512,1024]
for fil in filters:
    convnet= conv_2d(convnet, fil, 3, strides=1,padding='same',activation='LeakyReLU',
                     regularizer=None,weight_decay=decay)                  
    convnet= max_pool_2d(convnet, 2, strides=2)

###########

convnet= conv_2d(convnet, 256, 3, strides=1,padding='same',activation='LeakyReLU',regularizer=None,weight_decay=decay)
convnet= conv_2d(convnet, 512, 3, strides=1,padding='same',activation='LeakyReLU',regularizer=None,weight_decay=decay)





#del convnet
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

filters=[32,64,128,64,32]

for i in filters:
    convnet = conv_2d(convnet, i, 5, activation='LeakyReLU')
    convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('\log\{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]


print(len(train))
print(train[1][0].shape)
print(X.shape)
print(len(test_x))
print(test_x[0].shape)
print(X.size)

In [ ]:
model.fit({'input': X}, {'targets': Y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

In [3]:
import os
os.sys.path

['D:\\cnntry',
 'C:\\Users\\Admin\\Anaconda3\\python37.zip',
 'C:\\Users\\Admin\\Anaconda3\\DLLs',
 'C:\\Users\\Admin\\Anaconda3\\lib',
 'C:\\Users\\Admin\\Anaconda3',
 '',
 'C:\\Users\\Admin\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\Admin\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Admin\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Admin\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Admin\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Admin\\.ipython']

In [ ]:
#for i in range(10): print(random.getrandbits(1))
#model.save(MODEL_NAME)

tflearn.layers.conv.conv_2d (incoming, nb_filter, filter_size, strides=1, padding='same', 
                             activation='linear', bias=True, weights_init='uniform_scaling', 
                             bias_init='zeros', regularizer=None, weight_decay=0.001, trainable=True, 
                             restore=True, reuse=False, scope=None, name='Conv2D')
Input= 4-D Tensor [batch, height, width, in_channels].
Output= 4-D Tensor [batch, new height, new width, nb_filter].


tflearn.layers.conv.max_pool_2d (incoming, kernel_size, strides=None, padding='same', name='MaxPool2D')
Input= 4-D Tensor [batch, height, width, in_channels].
Output= 4-D Tensor [batch, pooled height, pooled width, in_channels].


tflearn.layers.core.fully_connected (incoming, n_units, activation='linear', 
                                     bias=True, weights_init='truncated_normal', bias_init='zeros', 
                                     regularizer=None, weight_decay=0.001, trainable=True, restore=True, 
                                     reuse=False, scope=None, name='FullyConnected')
Input= (2+)-D Tensor [samples, input dim]. If not 2D, input will be flatten.
Output= 2D Tensor [samples, n_units].


tflearn.layers.estimator.regression (incoming, placeholder='default', optimizer='adam', 
                                     loss='categorical_crossentropy', metric='default', learning_rate=0.001, 
                                     dtype=tf.float32, batch_size=64, shuffle_batches=True, to_one_hot=False, 
                                     n_classes=None, trainable_vars=None, restore=True, op_name=None, 
                                     validation_monitors=None, validation_batch_size=None, name=None)
                                     
tflearn.activations.leaky_relu (x, alpha=0.1, name='LeakyReLU')